# Imports

In [1]:
import osmnx as ox, networkx as nx, geopandas as gpd, matplotlib.pyplot as plt
from shapely.geometry import Point, LineString, Polygon, MultiPolygon
from descartes import PolygonPatch
import shapely
import pandas as pd
import numpy as np
ox.config(log_console=True, use_cache=True)

In [ ]:
amenities = pd.read_csv("safegraphplaces_best.csv")
APW = gpd.read_file("airportwayparcelsfinal/airportwayparcels.shp")

amenities['geometry'] = amenities.apply(lambda z: Point(z.longitude, z.latitude), axis=1)
amenities = gpd.GeoDataFrame(amenities)

amenities.crs = {'init' :'epsg:4326'}
#amenities = amenities.to_crs(epsg = 4326)
APW = APW.to_crs(epsg = 4326)

In [ ]:
#Stockton Blocks
BGs = gpd.read_file('blocks/Census2010Block.shp')
BGs = BGs.to_crs(epsg = 4326)

city = ox.gdf_from_place('Stockton, California, USA')

#Generate geodataframe of blocks WITHIN the city of stockton
BGs = gpd.overlay(BGs, city, how='intersection')

# Create individual matrices by travel time

In [ ]:
blockMatrix = pd.DataFrame(index = amenities.safegraph_place_id, columns=BGs.GEOID10)

In [ ]:
for i in range(0,len(APW)):
    parcel = APW.ix[i,:]
    blockMatrix.loc[parcel.APN] = list(blockMatrix.head(1).values[0])

In [ ]:
blockMatrix_car = blockMatrix.copy()
blockMatrix_bike = blockMatrix.copy()
blockMatrix_walk = blockMatrix.copy()
blockMatrix_transit = blockMatrix.copy()

In [ ]:
columns = list(blockMatrix.columns.values)

In [ ]:
index_vals = list(blockMatrix.index)

In [ ]:
isochrone_polys_drive = pd.read_csv('isochrone_polys_car.csv')

In [ ]:
#start_time = timeit.default_timer()
#consider spatial join/intersection
trip_times = [1, 3, 5, 10, 15, 20, 25, 30]
count = 0
weirdBlocks = []
for i in range(0,len(isochrone_polys_drive)):
    #Read in isochrones, amenities and parcels
    amenities_GEO = gpd.GeoDataFrame(amenities.geometry, geometry=amenities.geometry)
    APW_GEO = gpd.GeoDataFrame(APW.geometry.centroid, geometry=APW.geometry.centroid)
    iso = isochrone_polys_drive.ix[i]
    iso0 = shapely.wkt.loads(iso[1])
    iso1 = shapely.wkt.loads(iso[2])
    iso2 = shapely.wkt.loads(iso[3])
    iso3 = shapely.wkt.loads(iso[4])
    iso4 = shapely.wkt.loads(iso[5])
    iso5 = shapely.wkt.loads(iso[6])
    iso6 = shapely.wkt.loads(iso[7])
    iso7 = shapely.wkt.loads(iso[8])
    #Create buffer rings from the polygons - catching errors where an isochrone fails
    try:
        block_SHAPE = gpd.GeoDataFrame(gpd.GeoSeries(MultiPolygon([iso0.symmetric_difference(iso1),
            iso1.symmetric_difference(iso2),
            iso2.symmetric_difference(iso3),
            iso3.symmetric_difference(iso4),
            iso4.symmetric_difference(iso5),
            iso5.symmetric_difference(iso6),
            iso6.symmetric_difference(iso7),iso7]).geoms), geometry=gpd.GeoSeries(MultiPolygon([iso0.symmetric_difference(iso1),
            iso1.symmetric_difference(iso2),
            iso2.symmetric_difference(iso3),
            iso3.symmetric_difference(iso4),
            iso4.symmetric_difference(iso5),
            iso5.symmetric_difference(iso6),
            iso6.symmetric_difference(iso7),iso7]).geoms))
    except:
        TypeError
        try:
            block_SHAPE = gpd.GeoDataFrame(gpd.GeoSeries(MultiPolygon([iso0.symmetric_difference(iso1),
                iso1.symmetric_difference(iso2),
                iso2.symmetric_difference(iso3),
                iso3.symmetric_difference(iso4),
                iso4.symmetric_difference(iso5),
                iso5.symmetric_difference(iso7),iso7]).geoms), geometry=gpd.GeoSeries(MultiPolygon([iso0.symmetric_difference(iso1),
                iso1.symmetric_difference(iso2),
                iso2.symmetric_difference(iso3),
                iso3.symmetric_difference(iso4),
                iso4.symmetric_difference(iso5),
                iso5.symmetric_difference(iso7),iso7]).geoms))
        except:
            TypeError
            block_SHAPE = gpd.GeoDataFrame(gpd.GeoSeries(MultiPolygon([iso0.symmetric_difference(iso1),
                iso1.symmetric_difference(iso2),
                iso2.symmetric_difference(iso3),
                iso3.symmetric_difference(iso4),
                iso4.symmetric_difference(iso7),iso7]).geoms), geometry=gpd.GeoSeries(MultiPolygon([iso0.symmetric_difference(iso1),
                iso1.symmetric_difference(iso2),
                iso2.symmetric_difference(iso3),
                iso3.symmetric_difference(iso4),
                iso4.symmetric_difference(iso7),iso7]).geoms))
    #Catch a unique driving isochrone issue - 30 + 25 minute isochrones are the same
    if iso0.area == iso1.area:
            block_SHAPE.index = block_SHAPE.index + 1
            print(i)
            print("SPECIAL CASE")
            weirdBlocks.append(i)
    #Spatial overlap
    spatial_op1 = list(gpd.sjoin(amenities_GEO, block_SHAPE, how="left", op='within').index_right)
    #print(spatial_op1)
    spatial_op2 = list(gpd.sjoin(APW_GEO, block_SHAPE, how="left", op='within').index_right)
    #print(spatial_op2)
    indexes = spatial_op1 + spatial_op2
    #print(indexes)
    #Add to index
    blockMatrix_car.ix[:,i] = indexes
    print(str(i) + " out of " + str(len(isochrone_polys_drive)))
#elapsed = timeit.default_timer() - start_time
#print(elapsed)

In [ ]:
blockMatrix_car.to_csv('blockMatrix_car.gz',compression='gzip')

In [ ]:
isochrone_polys_bike = pd.read_csv('isochrone_polys_bike.csv')

In [ ]:
#start_time = timeit.default_timer()
#consider spatial join/intersection
#SEE COMMENTS ABOVE FOR PROCEDURE
trip_times = [1, 3, 5, 10, 15, 20, 25, 30]
count = 0
for i in range(0,len(isochrone_polys_bike)):
    amenities_GEO = gpd.GeoDataFrame(amenities.geometry, geometry=amenities.geometry)
    APW_GEO = gpd.GeoDataFrame(APW.geometry.centroid, geometry=APW.geometry.centroid)
    iso = isochrone_polys_bike.ix[i]
    iso0 = shapely.wkt.loads(iso[1])
    iso1 = shapely.wkt.loads(iso[2])
    iso2 = shapely.wkt.loads(iso[3])
    iso3 = shapely.wkt.loads(iso[4])
    iso4 = shapely.wkt.loads(iso[5])
    iso5 = shapely.wkt.loads(iso[6])
    iso6 = shapely.wkt.loads(iso[7])
    iso7 = shapely.wkt.loads(iso[8])
    try:
        block_SHAPE = gpd.GeoDataFrame(gpd.GeoSeries(MultiPolygon([iso0.symmetric_difference(iso1),
            iso1.symmetric_difference(iso2),
            iso2.symmetric_difference(iso3),
            iso3.symmetric_difference(iso4),
            iso4.symmetric_difference(iso5),
            iso5.symmetric_difference(iso6),
            iso6.symmetric_difference(iso7), iso7]).geoms), geometry=gpd.GeoSeries(MultiPolygon([iso0.symmetric_difference(iso1),
            iso1.symmetric_difference(iso2),
            iso2.symmetric_difference(iso3),
            iso3.symmetric_difference(iso4),
            iso4.symmetric_difference(iso5),
            iso5.symmetric_difference(iso6),
            iso6.symmetric_difference(iso7), iso7]).geoms))
    except:
        TypeError
        try:
            block_SHAPE = gpd.GeoDataFrame(gpd.GeoSeries(MultiPolygon([iso0.symmetric_difference(iso1),
                iso1.symmetric_difference(iso2),
                iso2.symmetric_difference(iso3),
                iso3.symmetric_difference(iso4),
                iso4.symmetric_difference(iso5),
                iso5.symmetric_difference(iso7), iso7]).geoms), geometry=gpd.GeoSeries(MultiPolygon([iso0.symmetric_difference(iso1),
                iso1.symmetric_difference(iso2),
                iso2.symmetric_difference(iso3),
                iso3.symmetric_difference(iso4),
                iso4.symmetric_difference(iso5),
                iso5.symmetric_difference(iso7), iso7]).geoms))
        except:
            TypeError
            block_SHAPE = gpd.GeoDataFrame(gpd.GeoSeries(MultiPolygon([iso0.symmetric_difference(iso1),
                iso1.symmetric_difference(iso2),
                iso2.symmetric_difference(iso3),
                iso3.symmetric_difference(iso4),
                iso4.symmetric_difference(iso7), iso7]).geoms), geometry=gpd.GeoSeries(MultiPolygon([iso0.symmetric_difference(iso1),
                iso1.symmetric_difference(iso2),
                iso2.symmetric_difference(iso3),
                iso3.symmetric_difference(iso4),
                iso4.symmetric_difference(iso7), iso7]).geoms))
        if iso0.area == iso1.area:
            block_SHAPE.index = block_SHAPE.index + 1
            print(i)
            print("SPECIAL CASE")
    spatial_op1 = list(gpd.sjoin(amenities_GEO, block_SHAPE, how="left", op='within').index_right)
    #print(spatial_op1)
    spatial_op2 = list(gpd.sjoin(APW_GEO, block_SHAPE, how="left", op='within').index_right)
    #print(spatial_op2)
    indexes = spatial_op1 + spatial_op2
    #print(indexes)
    blockMatrix_bike.ix[:,i] = indexes
    print(str(i) + " out of " + str(len(isochrone_polys_bike)))
#elapsed = timeit.default_timer() - start_time
#print(elapsed)

In [ ]:
blockMatrix_bike.to_csv('blockMatrix_bike.gz',compression='gzip')

In [ ]:
isochrone_polys_walk = pd.read_csv('isochrone_polys_walk.csv')

In [ ]:
#start_time = timeit.default_timer()
#consider spatial join/intersection
#SEE COMMENTS ABOVE FOR PROCEDURE
trip_times = [1, 3, 5, 10, 15, 20, 25, 30]
count = 0
for i in range(0,2483):
    amenities_GEO = gpd.GeoDataFrame(amenities.geometry, geometry=amenities.geometry)
    APW_GEO = gpd.GeoDataFrame(APW.geometry.centroid, geometry=APW.geometry.centroid)
    iso = isochrone_polys_walk.ix[i]
    iso0 = shapely.wkt.loads(iso[1])
    iso1 = shapely.wkt.loads(iso[2])
    iso2 = shapely.wkt.loads(iso[3])
    iso3 = shapely.wkt.loads(iso[4])
    iso4 = shapely.wkt.loads(iso[5])
    iso5 = shapely.wkt.loads(iso[6])
    iso6 = shapely.wkt.loads(iso[7])
    iso7 = shapely.wkt.loads(iso[8])
    try:
        block_SHAPE = gpd.GeoDataFrame(gpd.GeoSeries(MultiPolygon([iso0.symmetric_difference(iso1),
            iso1.symmetric_difference(iso2),
            iso2.symmetric_difference(iso3),
            iso3.symmetric_difference(iso4),
            iso4.symmetric_difference(iso5),
            iso5.symmetric_difference(iso6),
            iso6.symmetric_difference(iso7), iso7]).geoms), geometry=gpd.GeoSeries(MultiPolygon([iso0.symmetric_difference(iso1),
            iso1.symmetric_difference(iso2),
            iso2.symmetric_difference(iso3),
            iso3.symmetric_difference(iso4),
            iso4.symmetric_difference(iso5),
            iso5.symmetric_difference(iso6),
            iso6.symmetric_difference(iso7), iso7]).geoms))
    except:
        TypeError
        try:
            block_SHAPE = gpd.GeoDataFrame(gpd.GeoSeries(MultiPolygon([iso0.symmetric_difference(iso1),
                iso1.symmetric_difference(iso2),
                iso2.symmetric_difference(iso3),
                iso3.symmetric_difference(iso4),
                iso4.symmetric_difference(iso5),
                iso5.symmetric_difference(iso7), iso7]).geoms), geometry=gpd.GeoSeries(MultiPolygon([iso0.symmetric_difference(iso1),
                iso1.symmetric_difference(iso2),
                iso2.symmetric_difference(iso3),
                iso3.symmetric_difference(iso4),
                iso4.symmetric_difference(iso5),
                iso5.symmetric_difference(iso7), iso7]).geoms))
        except:
            TypeError
            block_SHAPE = gpd.GeoDataFrame(gpd.GeoSeries(MultiPolygon([iso0.symmetric_difference(iso1),
                iso1.symmetric_difference(iso2),
                iso2.symmetric_difference(iso3),
                iso3.symmetric_difference(iso4),
                iso4.symmetric_difference(iso7), iso7]).geoms), geometry=gpd.GeoSeries(MultiPolygon([iso0.symmetric_difference(iso1),
                iso1.symmetric_difference(iso2),
                iso2.symmetric_difference(iso3),
                iso3.symmetric_difference(iso4),
                iso4.symmetric_difference(iso7), iso7]).geoms))
    spatial_op1 = list(gpd.sjoin(amenities_GEO, block_SHAPE, how="left", op='within').index_right)
    #print(spatial_op1)
    spatial_op2 = list(gpd.sjoin(APW_GEO, block_SHAPE, how="left", op='within').index_right)
    #print(spatial_op2)
    indexes = spatial_op1 + spatial_op2
    #print(indexes)
    blockMatrix_walk.ix[:,i] = indexes
    print(str(i) + " out of " + str(len(isochrone_polys_walk)))
#elapsed = timeit.default_timer() - start_time
#print(elapsed)

In [ ]:
#Blocks where the isochrones are all points/lines
blockMatrix_walk.ix[:,2483] = -1
blockMatrix_walk.ix[:,2484] = -1
blockMatrix_walk.ix[:,2906] = -1
blockMatrix_walk.ix[:,2907] = -1
blockMatrix_walk.ix[:,4359] = -1
blockMatrix_walk.ix[:,4360] = -1
blockMatrix_walk.ix[:,4361] = -1

In [ ]:
#start_time = timeit.default_timer()
#consider spatial join/intersection
trip_times = [1, 3, 5, 10, 15, 20, 25, 30]
count = 0
for i in range(2485,2906):
    amenities_GEO = gpd.GeoDataFrame(amenities.geometry, geometry=amenities.geometry)
    APW_GEO = gpd.GeoDataFrame(APW.geometry.centroid, geometry=APW.geometry.centroid)
    iso = isochrone_polys_walk.ix[i]
    iso0 = shapely.wkt.loads(iso[1])
    iso1 = shapely.wkt.loads(iso[2])
    iso2 = shapely.wkt.loads(iso[3])
    iso3 = shapely.wkt.loads(iso[4])
    iso4 = shapely.wkt.loads(iso[5])
    iso5 = shapely.wkt.loads(iso[6])
    iso6 = shapely.wkt.loads(iso[7])
    iso7 = shapely.wkt.loads(iso[8])
    try:
        block_SHAPE = gpd.GeoDataFrame(gpd.GeoSeries(MultiPolygon([iso0.symmetric_difference(iso1),
            iso1.symmetric_difference(iso2),
            iso2.symmetric_difference(iso3),
            iso3.symmetric_difference(iso4),
            iso4.symmetric_difference(iso5),
            iso5.symmetric_difference(iso6),
            iso6.symmetric_difference(iso7), iso7]).geoms), geometry=gpd.GeoSeries(MultiPolygon([iso0.symmetric_difference(iso1),
            iso1.symmetric_difference(iso2),
            iso2.symmetric_difference(iso3),
            iso3.symmetric_difference(iso4),
            iso4.symmetric_difference(iso5),
            iso5.symmetric_difference(iso6),
            iso6.symmetric_difference(iso7), iso7]).geoms))
    except:
        TypeError
        try:
            block_SHAPE = gpd.GeoDataFrame(gpd.GeoSeries(MultiPolygon([iso0.symmetric_difference(iso1),
                iso1.symmetric_difference(iso2),
                iso2.symmetric_difference(iso3),
                iso3.symmetric_difference(iso4),
                iso4.symmetric_difference(iso5),
                iso5.symmetric_difference(iso7), iso7]).geoms), geometry=gpd.GeoSeries(MultiPolygon([iso0.symmetric_difference(iso1),
                iso1.symmetric_difference(iso2),
                iso2.symmetric_difference(iso3),
                iso3.symmetric_difference(iso4),
                iso4.symmetric_difference(iso5),
                iso5.symmetric_difference(iso7), iso7]).geoms))
        except:
            TypeError
            block_SHAPE = gpd.GeoDataFrame(gpd.GeoSeries(MultiPolygon([iso0.symmetric_difference(iso1),
                iso1.symmetric_difference(iso2),
                iso2.symmetric_difference(iso3),
                iso3.symmetric_difference(iso4),
                iso4.symmetric_difference(iso7), iso7]).geoms), geometry=gpd.GeoSeries(MultiPolygon([iso0.symmetric_difference(iso1),
                iso1.symmetric_difference(iso2),
                iso2.symmetric_difference(iso3),
                iso3.symmetric_difference(iso4),
                iso4.symmetric_difference(iso7), iso7]).geoms))
    spatial_op1 = list(gpd.sjoin(amenities_GEO, block_SHAPE, how="left", op='within').index_right)
    #print(spatial_op1)
    spatial_op2 = list(gpd.sjoin(APW_GEO, block_SHAPE, how="left", op='within').index_right)
    #print(spatial_op2)
    indexes = spatial_op1 + spatial_op2
    #print(indexes)
    blockMatrix_walk.ix[:,i] = indexes
    print(str(i) + " out of " + str(len(isochrone_polys_walk)))
#elapsed = timeit.default_timer() - start_time
#print(elapsed)

In [ ]:
#start_time = timeit.default_timer()
#consider spatial join/intersection
trip_times = [1, 3, 5, 10, 15, 20, 25, 30]
count = 0
for i in range(2908,4359):
    amenities_GEO = gpd.GeoDataFrame(amenities.geometry, geometry=amenities.geometry)
    APW_GEO = gpd.GeoDataFrame(APW.geometry.centroid, geometry=APW.geometry.centroid)
    iso = isochrone_polys_walk.ix[i]
    iso0 = shapely.wkt.loads(iso[1])
    iso1 = shapely.wkt.loads(iso[2])
    iso2 = shapely.wkt.loads(iso[3])
    iso3 = shapely.wkt.loads(iso[4])
    iso4 = shapely.wkt.loads(iso[5])
    iso5 = shapely.wkt.loads(iso[6])
    iso6 = shapely.wkt.loads(iso[7])
    iso7 = shapely.wkt.loads(iso[8])
    try:
        block_SHAPE = gpd.GeoDataFrame(gpd.GeoSeries(MultiPolygon([iso0.symmetric_difference(iso1),
            iso1.symmetric_difference(iso2),
            iso2.symmetric_difference(iso3),
            iso3.symmetric_difference(iso4),
            iso4.symmetric_difference(iso5),
            iso5.symmetric_difference(iso6),
            iso6.symmetric_difference(iso7), iso7]).geoms), geometry=gpd.GeoSeries(MultiPolygon([iso0.symmetric_difference(iso1),
            iso1.symmetric_difference(iso2),
            iso2.symmetric_difference(iso3),
            iso3.symmetric_difference(iso4),
            iso4.symmetric_difference(iso5),
            iso5.symmetric_difference(iso6),
            iso6.symmetric_difference(iso7), iso7]).geoms))
    except:
        TypeError
        try:
            block_SHAPE = gpd.GeoDataFrame(gpd.GeoSeries(MultiPolygon([iso0.symmetric_difference(iso1),
                iso1.symmetric_difference(iso2),
                iso2.symmetric_difference(iso3),
                iso3.symmetric_difference(iso4),
                iso4.symmetric_difference(iso5),
                iso5.symmetric_difference(iso7), iso7]).geoms), geometry=gpd.GeoSeries(MultiPolygon([iso0.symmetric_difference(iso1),
                iso1.symmetric_difference(iso2),
                iso2.symmetric_difference(iso3),
                iso3.symmetric_difference(iso4),
                iso4.symmetric_difference(iso5),
                iso5.symmetric_difference(iso7), iso7]).geoms))
        except:
            TypeError
            block_SHAPE = gpd.GeoDataFrame(gpd.GeoSeries(MultiPolygon([iso0.symmetric_difference(iso1),
                iso1.symmetric_difference(iso2),
                iso2.symmetric_difference(iso3),
                iso3.symmetric_difference(iso4),
                iso4.symmetric_difference(iso7), iso7]).geoms), geometry=gpd.GeoSeries(MultiPolygon([iso0.symmetric_difference(iso1),
                iso1.symmetric_difference(iso2),
                iso2.symmetric_difference(iso3),
                iso3.symmetric_difference(iso4),
                iso4.symmetric_difference(iso7), iso7]).geoms))
    spatial_op1 = list(gpd.sjoin(amenities_GEO, block_SHAPE, how="left", op='within').index_right)
    #print(spatial_op1)
    spatial_op2 = list(gpd.sjoin(APW_GEO, block_SHAPE, how="left", op='within').index_right)
    #print(spatial_op2)
    indexes = spatial_op1 + spatial_op2
    #print(indexes)
    blockMatrix_walk.ix[:,i] = indexes
    print(str(i) + " out of " + str(len(isochrone_polys_walk)))
#elapsed = timeit.default_timer() - start_time
#print(elapsed)

In [ ]:
#start_time = timeit.default_timer()
#consider spatial join/intersection
trip_times = [1, 3, 5, 10, 15, 20, 25, 30]
count = 0
for i in range(4362,len(isochrone_polys_walk)):
    amenities_GEO = gpd.GeoDataFrame(amenities.geometry, geometry=amenities.geometry)
    APW_GEO = gpd.GeoDataFrame(APW.geometry.centroid, geometry=APW.geometry.centroid)
    iso = isochrone_polys_walk.ix[i]
    iso0 = shapely.wkt.loads(iso[1])
    iso1 = shapely.wkt.loads(iso[2])
    iso2 = shapely.wkt.loads(iso[3])
    iso3 = shapely.wkt.loads(iso[4])
    iso4 = shapely.wkt.loads(iso[5])
    iso5 = shapely.wkt.loads(iso[6])
    iso6 = shapely.wkt.loads(iso[7])
    iso7 = shapely.wkt.loads(iso[8])
    try:
        block_SHAPE = gpd.GeoDataFrame(gpd.GeoSeries(MultiPolygon([iso0.symmetric_difference(iso1),
            iso1.symmetric_difference(iso2),
            iso2.symmetric_difference(iso3),
            iso3.symmetric_difference(iso4),
            iso4.symmetric_difference(iso5),
            iso5.symmetric_difference(iso6),
            iso6.symmetric_difference(iso7), iso7]).geoms), geometry=gpd.GeoSeries(MultiPolygon([iso0.symmetric_difference(iso1),
            iso1.symmetric_difference(iso2),
            iso2.symmetric_difference(iso3),
            iso3.symmetric_difference(iso4),
            iso4.symmetric_difference(iso5),
            iso5.symmetric_difference(iso6),
            iso6.symmetric_difference(iso7), iso7]).geoms))
    except:
        TypeError
        try:
            block_SHAPE = gpd.GeoDataFrame(gpd.GeoSeries(MultiPolygon([iso0.symmetric_difference(iso1),
                iso1.symmetric_difference(iso2),
                iso2.symmetric_difference(iso3),
                iso3.symmetric_difference(iso4),
                iso4.symmetric_difference(iso5),
                iso5.symmetric_difference(iso7), iso7]).geoms), geometry=gpd.GeoSeries(MultiPolygon([iso0.symmetric_difference(iso1),
                iso1.symmetric_difference(iso2),
                iso2.symmetric_difference(iso3),
                iso3.symmetric_difference(iso4),
                iso4.symmetric_difference(iso5),
                iso5.symmetric_difference(iso7), iso7]).geoms))
        except:
            TypeError
            block_SHAPE = gpd.GeoDataFrame(gpd.GeoSeries(MultiPolygon([iso0.symmetric_difference(iso1),
                iso1.symmetric_difference(iso2),
                iso2.symmetric_difference(iso3),
                iso3.symmetric_difference(iso4),
                iso4.symmetric_difference(iso7), iso7]).geoms), geometry=gpd.GeoSeries(MultiPolygon([iso0.symmetric_difference(iso1),
                iso1.symmetric_difference(iso2),
                iso2.symmetric_difference(iso3),
                iso3.symmetric_difference(iso4),
                iso4.symmetric_difference(iso7), iso7]).geoms))
    spatial_op1 = list(gpd.sjoin(amenities_GEO, block_SHAPE, how="left", op='within').index_right)
    #print(spatial_op1)
    spatial_op2 = list(gpd.sjoin(APW_GEO, block_SHAPE, how="left", op='within').index_right)
    #print(spatial_op2)
    indexes = spatial_op1 + spatial_op2
    #print(indexes)
    blockMatrix_walk.ix[:,i] = indexes
    print(str(i) + " out of " + str(len(isochrone_polys_walk)))
#elapsed = timeit.default_timer() - start_time
#print(elapsed)

In [ ]:
blockMatrix_walk.to_csv('blockMatrix_walk.gz', compression='gzip')

In [ ]:
#Sorting algorithm for transit isochrone dataset
def sort(geo):
    if iso1.contains(geo):
        return 7
    elif iso3.contains(geo):
        return 6
    elif iso5.contains(geo):
        return 5
    elif iso10.contains(geo):
        return 4
    elif iso15.contains(geo):
        return 3
    elif iso20.contains(geo):
        return 2
    elif iso25.contains(geo):
        return 1
    elif iso30.contains(geo):
        return 0
    else:
        return 10

In [ ]:
isochrones_polys_transit = pd.read_csv('transitIsochrones_12132018.csv')

In [ ]:
#start_time = timeit.default_timer()
#consider spatial join/intersection
#SEE COMMENTS ABOVE FOR PROCEDURE - SORTING APPROACH IS SLIGHTLY DIFFERENT, see fn above
trip_times = [1, 3, 5, 10, 15, 20, 25, 30]
count = 0
amenities_GEO = gpd.GeoDataFrame(amenities.geometry, geometry=amenities.geometry)
APW_GEO = gpd.GeoDataFrame(APW.geometry.centroid, geometry=APW.geometry.centroid)
for geoid in isochrones_polys_transit.index.unique():
    isochrones = isochrones_polys_transit.ix[geoid,:]
    iso1 = shapely.wkt.loads(isochrones['1'])
    iso3 = shapely.wkt.loads(isochrones['3'])
    iso5 = shapely.wkt.loads(isochrones['5'])
    iso10 = shapely.wkt.loads(isochrones['10'])
    iso15 = shapely.wkt.loads(isochrones['15'])
    iso20 = shapely.wkt.loads(isochrones['20'])
    iso25 = shapely.wkt.loads(isochrones['25'])
    iso30 = shapely.wkt.loads(isochrones['30'])
    spatial_op1 = list(amenities_GEO.geometry.apply(sort))
    spatial_op2 = list(APW_GEO.geometry.apply(sort))
    indexes = spatial_op1 + spatial_op2
    #print(indexes)
    blockMatrix_transit.ix[:,('0' + str(geoid))] = indexes
    print(str(count) + " out of " + str(len(isochrones_polys_transit)))
    count += 1
#elapsed = timeit.default_timer() - start_time
#print(elapsed)

In [ ]:
blockMatrix_transit.to_csv('blockMatrix_transit.gz', compression='gzip')

# Join to create full OD matrix

In [2]:
#Haven't incorporated this yet but should be relatively easy
type_id_set = pd.read_csv('SafeGraphPlacesTypes.csv')

In [ ]:
#To add types to the full stack
test = pd.DataFrame(blockMatrix_bike.ix[:,-1])
for i in range(0,4501):
    test = test.append(pd.DataFrame(blockMatrix_bike.ix[:,-1]))

In [ ]:
#Function to convert classes assigned above to actual travel times
def filterstoTimes(x):
    if x == 0:
        return (30+25)/2
    elif x == 1:
        return (20+25)/2
    elif x == 2:
        return (20+15)/2
    elif x == 3:
        return (15 + 10)/2
    elif x == 4:
        return (10 + 5) / 2
    elif x == 5:
        return (5+ 3) / 2
    elif x == 6:
        return 2
    elif x == 7:
        return 0.5
    else:
        return 45

In [ ]:
#convert categories to times - check dataset before applying to be sure the map is applied to the right subset
blockMatrix_bike = blockMatrix_bike.applymap(filterstoTimes)
blockMatrix_car = blockMatrix_car.applymap(filterstoTimes)
blockMatrix_walk = blockMatrix_walk.applymap(filterstoTimes)
blockMatrix_transit = blockMatrix_transit.applymap(filterstoTimes)

In [ ]:
blockMatrix_bike.to_csv('blockMatrix_bike.gz',compression='gzip')
blockMatrix_car.to_csv('blockMatrix_car.gz',compression='gzip')
blockMatrix_walk.to_csv('blockMatrix_walk.gz',compression='gzip')
blockMatrix_transit.to_csv('blockMatrix_transit.gz',compression='gzip')

Note: this may have to be done incrementally to avoid memory/ram errors - write out as frequently as possible

In [ ]:
fullStack = car_stack[['AmenityID', 'BlockID', 'carTime']]
fullStack = fullStack.sort_values(["AmenityID", "BlockID"])

In [ ]:
walk_stack = blockMatrix_walk.stack().reset_index()
walk_stack.columns = ['AmenityID', "BlockID",'WalkTime']
walk_stack = walk_stack.sort_values(["AmenityID", "BlockID"])

In [6]:
bike_stack = blockMatrix_bike.stack().reset_index()
bike_stack.columns = ['AmenityID', "BlockID",'BikeTime']
bike_stack = bike_stack.sort_values(["AmenityID", "BlockID"])

In [ ]:
transit_stack = blockMatrix_transit.stack().reset_index()
transit_stack.columns = ['AmenityID', "BlockID",'TransitTime']
transit_stack = transit_stack.sort_values(["AmenityID", "BlockID"])

In [ ]:
fullStack['WalkTime'] = walk_stack.WalkTime

In [ ]:
fullStack['BikeTime'] = bike_stack.BikeTime

In [ ]:
fullStack['TransitTime'] = transit_stack.transitTime

In [ ]:
#adding types
